<a href="https://colab.research.google.com/github/sufaye2024/bhban_rpa/blob/master/2024_DH_%EA%B2%A8%EC%9A%B8%ED%95%99%EA%B5%90_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 역사 문서에 토픽 모델링 적용해보기
토픽 모델링을 사용해 세종 실록의 내용을 개략적으로 살펴보는 연습을 합니다.
전체 3만건이 넘는 세종 실록의 기사들이 대체로 어떤 주제들을 다루고 있는지 살펴보고, 각 주제를 다루는 문서에는 어떤 것이 있는지 확인해보는 방법을 익힙니다.

또한 이 실습에서는 엑셀로 구성된 데이터 파일을 읽고 쓰는 방법까지도 다룹니다.


In [ ]:
!pip install kiwipiepy tomotopy nltk pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813256 sha256=6f891a0c4939df7ca82d0b4a6b7bddd1711223a276d65cd8c6dfca82bd9f5104
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [ ]:
import kiwipiepy
import tomotopy as tp
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 엑셀 시트에 있는 데이터셋 읽어오기

In [ ]:
# 구글 드라이브에서 sejong_silok.xlsx를 가져옵니다.
# 혹은 https://drive.google.com/file/d/1YFxreuUzvipwNSEX5Yy643Cw78zWoOSI/view?usp=drive_link 에서 파일을 다운받은뒤
# 직접 업로드하는 방법도 가능합니다.
FILEID = "1YFxreuUzvipwNSEX5Yy643Cw78zWoOSI"
FILENAME = "sejong_silok.xlsx"

!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILEID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILEID}" -O {FILENAME} && rm -rf ~/cookies.txt

--2024-01-14 16:08:04--  https://docs.google.com/uc?export=download&confirm=&id=1YFxreuUzvipwNSEX5Yy643Cw78zWoOSI
Resolving docs.google.com (docs.google.com)... 64.233.188.101, 64.233.188.113, 64.233.188.102, ...
Connecting to docs.google.com (docs.google.com)|64.233.188.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1YFxreuUzvipwNSEX5Yy643Cw78zWoOSI&export=download [following]
--2024-01-14 16:08:05--  https://drive.usercontent.google.com/download?id=1YFxreuUzvipwNSEX5Yy643Cw78zWoOSI&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.170.132, 2404:6800:4008:c19::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.170.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12965824 (12M) [application/octet-stream]
Saving to: ‘sejong_silok.xlsx’

sejong_silok.xlsx   100%[===================>]  12.36M 

In [ ]:
# pd.read_excel을 이용해서 엑셀 파일에 있는 데이터를 읽어옵니다.
df = pd.read_excel('sejong_silok.xlsx')

In [ ]:
# 잘 읽어와졌는지 출력해봅니다.
df

,연도,월,일,번호,제목,본문
0,0,8,11,1,근정전에서 즉위 교서를 반포하다,"임금이 {근정전}에 나아가 교서를 반포하기를, ""삼가 생각하건대, {태조}께서 홍..."
1,0,8,11,2,예조에서 상왕전에 전(箋)을 올리는 절차를 아뢰다. 임금이 본궁으로 돌아오다,"예조에서 임금이 친히 상왕전(上王殿)에 나아가 전(箋)을 올리는 거동에 대하여, 그..."
2,0,8,11,3,세자의 사빈을 겸했던 박은·이원·유창 등에게 관직을 제수하다,"{박은}을 좌의정 영경연사(左議政領經筵事)로, {이원(李原)}을 우의정 영경연사(..."
3,0,8,12,1,전위를 받았음을 종묘에 고하다,"{이원}을 보내어 {종묘}에 고하기를, ""공을 쌓고 인(仁)을 쌓아 나라를 세우..."
4,0,8,12,2,당상관들과 의논하여 관직을 제수하다,"임금이 {하연(河演)}에게 이르기를, ""내가 인물을 잘 알지 못하니, 좌의정·우의..."
...,...,...,...,...,...,...
30942,32,2,16,1,일본국 사신 중 경능이 원의성의 글을 가지고 와서 토산물을 바치다,"{일본국} 사신 중[僧] {경능(敬楞)}이 왔는데, 그 글에 이르기를, ""{일본..."
30943,32,2,16,2,임금의 병환이 위독하여 정근을 정지하다,임금이 병환이 위독하여 정근(精勤)을 정지하였다.
30944,32,2,17,1,임금이 영응 대군 집 동별궁에서 훙하다,임금이 {영응 대군(永膺大君)} 집 동별궁(東別宮)에서 훙(薨)하였다. 【처음에 영...
30945,32,2,22,1,지중추원사 이선 등을 북경에 보내 부고를 고하고 시호를 청하다,지중추원사(知中樞院事) {이선(李渲)} 등을 {북경}에 보내어 부고를 고하고 시호...


# 형태소 분석하기

In [ ]:
kiwi = kiwipiepy.Kiwi(num_workers=2)

In [ ]:
# Kiwi 형태소 분석기 기반의 토크나이저입니다.
class KoreanTokenizer:
    def __init__(self, kiwi, allowed_tags:list = None, banned_tags:list = None, stopwords:list = None):
        self.kiwi = kiwi
        assert allowed_tags is None or banned_tags is None, 'allowed_tags and banned_tags cannot be used at the same time.'
        self.allowed_tags = set(allowed_tags) if allowed_tags is not None else None
        self.banned_tags = set(banned_tags) if banned_tags is not None else None
        self.stopwords = set(stopwords or [])

    def __call__(self, texts):
        for tokens in self.kiwi.tokenize(text for text, _ in texts):
            if self.allowed_tags is not None:
                tokens = [(token.tagged_form, token.start, token.len) for token in tokens if token.tag in self.allowed_tags and token.tagged_form not in self.stopwords]
            elif self.banned_tags is not None:
                tokens = [(token.tagged_form, token.start, token.len) for token in tokens if token.tag not in self.banned_tags and token.tagged_form not in self.stopwords]
            else:
                tokens = [(token.tagged_form, token.start, token.len) for token in tokens if token.tagged_form not in self.stopwords]
            yield tokens

In [ ]:
# "{근정전}", "상왕전(上王殿)"과 같은 패턴에서 한글로 된 부분만 선택해내기 위해 정규표현식을 사용합니다.
kiwi.add_re_word(r'\{([^"\'}()]+)\}', lambda m:(m.group(1), 'NNP', m.span(1)[0] - m.span(0)[0], m.span(1)[1] - m.span(0)[0]))
kiwi.add_re_word(r'(?<!\{)([^"\'() ]+)\([^)]+\)', lambda m:(m.group(1).strip().replace(' ', '_'), 'NNP', m.span(1)[0] - m.span(0)[0], m.span(1)[1] - m.span(0)[0]))
kiwi.add_re_word(r'\{([^"\'}(]+)\([^})]+\)\}', lambda m:(m.group(1).strip().replace(' ', '_'), 'NNP', m.span(1)[0] - m.span(0)[0], m.span(1)[1] - m.span(0)[0]))

# 형태소 분석기의 품사 태그에 관해서는
# https://github.com/bab2min/kiwipiepy?tab=readme-ov-file#%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8
# 를 참조해주세요.
tokenizer = KoreanTokenizer(
    kiwi,
    allowed_tags=['NNG', 'NNP',
                  'VV', 'VA',
                  'XR',
                  'SN', 'SL', 'SH'],
    stopwords=['하/VV', '되/VV', '이/MM', '그/MM', '저/MM']
)

In [ ]:
tokenized_samples = list(tokenizer((title + '\n' + content, None) for title, content in zip(df['제목'].head(10), df['본문'].head(10))))

In [ ]:
tokenized_samples

[[('근정전/NNP', 0, 3),
  ('즉위/NNG', 6, 2),
  ('교서/NNG', 9, 2),
  ('반포/NNG', 13, 2),
  ('임금/NNG', 18, 2),
  ('근정전/NNP', 22, 5),
  ('나아가/VV', 29, 3),
  ('교서/NNG', 33, 2),
  ('반포/NNG', 37, 2),
  ('생각/NNG', 49, 2),
  ('태조/NNP', 56, 4),
  ('홍업/NNP', 63, 6),
  ('초창/NNG', 71, 2),
  ('부왕/NNG', 77, 2),
  ('전하/NNG', 80, 2),
  ('크/VA', 85, 1),
  ('사업/NNG', 87, 2),
  ('삼가/VV', 99, 2),
  ('조심/NNG', 103, 2),
  ('하늘/NNG', 108, 2),
  ('공경/NNG', 112, 2),
  ('백성/NNG', 117, 2),
  ('사랑/NNG', 121, 2),
  ('충성/NNG', 127, 2),
  ('천자/NNP', 131, 6),
  ('효/NNG', 145, 1),
  ('공경/NNG', 149, 2),
  ('신명/NNP', 154, 6),
  ('통하/VV', 162, 2),
  ('나라/NNG', 166, 2),
  ('안팎/NNG', 170, 2),
  ('다스리/VV', 174, 3),
  ('평안/NNG', 178, 2),
  ('나라/NNG', 183, 2),
  ('창고/NNG', 187, 2),
  ('넉넉/XR', 191, 2),
  ('가득/NNG', 196, 2),
  ('해구/NNP', 202, 6),
  ('가/NNG', 208, 1),
  ('오/VV', 210, 1),
  ('복종/NNG', 213, 2),
  ('문치/NNP', 219, 6),
  ('융성/NNG', 227, 2),
  ('무위/NNP', 232, 6),
  ('떨치/VV', 240, 2),
  ('물/NNG', 247, 1),
  ('들리/VV', 250, 2

In [ ]:
# 토픽 모델링에 사용할 문서들 전체를 Corpus로 묶습니다.
corpus = tp.utils.Corpus(tokenizer=tokenizer)
# 제목과 본문을 합쳐서 하나의 텍스트로 만들어서 입력합니다. (\n는 줄바꿈 문자를 뜻합니다.)
corpus.process((title + '\n' + content, None) for title, content in zip(tqdm(df['제목']), df['본문']))

100%|██████████| 30947/30947 [06:54<00:00, 74.58it/s]


30947

In [ ]:
model = tp.LDAModel(k=10, min_df=5, tw=tp.TermWeight.ONE, rm_top=10, corpus=corpus)

In [ ]:
model.train(0)
model.summary(topic_word_top_n=10)

<Basic Info>
| LDAModel (current version: 0.12.7)
| 30947 docs, 1782265 words
| Total Vocabs: 74945, Used Vocabs: 20408
| Entropy of words: 8.20178
| Entropy of term-weighted words: 8.20178
| Removed Vocabs: 을/NNG 말/NNG 없/VA 사람/NNG 있/VV 아뢰/VV 일/NNG 임금/NNG 가/VV 따르/VV
|
<Training Info>
| Iterations: 0, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -11.05775
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 5 (minimum document frequency of words)
| rm_top: 10 (the number of top words to be removed)
| k: 10 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 4255832109 (random seed)
| trained in version 0.12.7
|
<Parameters>
| alpha (Dirichlet pr

In [ ]:
model.train(1000, workers=2, show_progress=True)

Iteration: 100%|██████████| 1000/1000 [02:08<00:00,  7.81it/s, LLPW: -8.136683]


In [ ]:
model.summary(topic_word_top_n=10)

<Basic Info>
| LDAModel (current version: 0.12.7)
| 30947 docs, 1782265 words
| Total Vocabs: 74945, Used Vocabs: 20408
| Entropy of words: 8.20178
| Entropy of term-weighted words: 8.20178
| Removed Vocabs: 을/NNG 말/NNG 없/VA 사람/NNG 있/VV 아뢰/VV 일/NNG 임금/NNG 가/VV 따르/VV
|
<Training Info>
| Iterations: 1000, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.13636
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 5 (minimum document frequency of words)
| rm_top: 10 (the number of top words to be removed)
| k: 10 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 4255832109 (random seed)
| trained in version 0.12.7
|
<Parameters>
| alpha (Dirichlet 

In [ ]:
# 학습이 부족했다는 판단이 들면 train을 추가로 더 수행하는 것도 좋은 생각입니다.
model.train(1000, workers=2, show_progress=True)

Iteration: 100%|██████████| 1000/1000 [02:05<00:00,  7.94it/s, LLPW: -8.136771]


In [ ]:
# 추가 train 이후 모델의 상태가 어떻게 달라졌는지 비교해봅시다.
model.summary(topic_word_top_n=10)

<Basic Info>
| LDAModel (current version: 0.12.7)
| 30947 docs, 1782265 words
| Total Vocabs: 74945, Used Vocabs: 20408
| Entropy of words: 8.20178
| Entropy of term-weighted words: 8.20178
| Removed Vocabs: 을/NNG 말/NNG 없/VA 사람/NNG 있/VV 아뢰/VV 일/NNG 임금/NNG 가/VV 따르/VV
|
<Training Info>
| Iterations: 2000, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.13674
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 5 (minimum document frequency of words)
| rm_top: 10 (the number of top words to be removed)
| k: 10 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 4255832109 (random seed)
| trained in version 0.12.7
|
<Parameters>
| alpha (Dirichlet 

In [ ]:
# Google Colab에서 Viewer를 실행시에는 다음의 print문에서 출력되는 주소로 접속해야 합니다.
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))
# 토픽 모델링 결과를 살펴볼 수 있는 뷰어를 8000포트에 띄웁니다.
tp.viewer.open_viewer(model, port=8000)

https://nzchla6hjpe-496ff2e9c6d22116-8000-colab.googleusercontent.com/
Serving a topic model viewer at http://127.0.0.1:8000/


127.0.0.1 - - [14/Jan/2024 16:22:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2024 16:22:36] code 404, message Not Found
127.0.0.1 - - [14/Jan/2024 16:22:36] "GET /favicon.ico HTTP/1.1" 404 -


KeyboardInterrupt: 

# 모델링 결과를 엑셀 시트로 저장하기
학습 결과가 마음에 들어 저장해두고 싶다면 다시 엑셀 시트로 내보내는 것도 가능합니다. 기존의 sejong_silok.xlsx에 문서별 주제 비율을 나타내는 열들을 추가하여 'sejong_silok_with_topics.xlsx' 라고 저장하고, 또 각 주제별 단어 분포들은 'sejong_silok_topic_word.xlsx' 라고 저장해봅시다.

In [ ]:
doc_topic = np.array([doc.get_topic_dist() for doc in model.docs])
for i, probs in enumerate(doc_topic.T):
  df[f'Topic #{i}'] = probs

In [ ]:
df.to_excel('sejong_silok_with_topics.xlsx')

In [ ]:
# 각 주제별로 상위 50개 단어를 출력하여 엑셀로 저장합니다.
topic_word_data = {}
for k in range(model.k):
  word_forms, word_probs = zip(*model.get_topic_words(k, top_n=50))
  topic_word_data[f'#{k}'] = word_forms
  topic_word_data[f'#{k} prob'] = word_probs

topic_word_df = pd.DataFrame(data=topic_word_data)

In [ ]:
topic_word_df.to_excel('sejong_silok_topic_word.xlsx')